<a href="https://colab.research.google.com/github/torzdf/faceswap/blob/colab/faceswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left">
  <a href="https://faceswap.dev"><img src="https://i.imgur.com/zHvjHnb.png"></img></a>Welcome to the FaceSwap Notebook
</p>



The FaceSwap Notebook is a means of running FaceSwap on Google Colab.

Colab will give you a free GPU for up to a 12 hour period (as long as your browser window remains open).

Please make sure you understand Google Colaboratory's terms of service. This notebook is for testing out Faceswap and experimenting. It is not for creating swaps on an industrial scale. [See here for more info](https://research.google.com/colaboratory/faq.html#gpu-availability).

At a minimum you must run the [Prerequisites](#scrollTo=GVzyxw4xkKI7). Once complete, you can jump to the section that your require.

**NB:** This notebook is provided as a courtesy. It is not a priority for the FaceSwap project so there is no guarantee that it will remain up to date and support is likely to be limited. Please be aware that this Notebook lacks some features from the full application.

>[Prerequisites](#scrollTo=GVzyxw4xkKI7)

>>[GPU Check](#scrollTo=_jYRKp5zkYMU)

>>[Link Google Drive](#scrollTo=Sb1aWvH3phjA)

>>[Get FaceSwap](#scrollTo=oql-3Tf7lquV)

>[Extract](#scrollTo=aWiYFqK4R64L)

>[Train](#scrollTo=o_M1CIMUr0hR)

>[Convert](#scrollTo=_Kp5gjPqr0_x)



# Prerequisites
First up we need to make sure we have a GPU instance and set up our environment.

Whatever task you plan to perform, you must run these steps first.

## GPU Check
Run the following code block to make sure you have been allocated a GPU:

In [0]:
#@title
!nvidia-smi

Wed Dec 11 02:49:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

 You should receive output similar to below:
```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+
```

If you have not been allocated a GPU then select `Runtime` > `Change runtime type` from the top menu and ensure that the `Runtime type` is `Python 3` and `Hardware Acceleration` is `GPU`

## Link Google Drive
Now we know we have a GPU instance, we need to link your Google Drive.

You will need to give Colab permissions to mount your drive, so run the code block below and follow the instructions to mount your drive:

In [0]:
#@title
import os
_ROOT="/content/drive/My Drive/_faceswap_colab"
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

if not os.path.exists(_ROOT):
  !mkdir "$_ROOT"

Mounted at /content/drive/


## Get FaceSwap
Time to tell Colab where to find/install the FaceSwap App. Select your options in the form, then hit play:

In [0]:
#@title App install location
#@markdown Where do you want Faceswap installed?
#@markdown * GoogleDrive: [Recommended] - Store the FaceSwap App on your Google
#@markdown Drive. This means any config files that are changed won't get lost
#@markdown when the instance destroys. However it will take up about 200MB of
#@markdown space.
#@markdown * Local: FaceSwap App will be stored in the Colab Notebook. The app
#@markdown will disappear when the instance dies.
location = "GoogleDrive"  #@param ['GoogleDrive', 'Local']
#@markdown Do You want to update an existing app or create a new one?
#@markdown * Update: [Recommended] - If the FaceSwap App is already installed in
#@markdown the chosen location, it will be updated. If the app doesn't already
#@markdown exist, it will be installed.
#@markdown * New: Uninstall any installed FaceSwap App and Reinstall for the
#@markdown chosen location.
app_update = "Update"  #@param ['New', 'Update']

import os
import shutil
_ROOT="/content/drive/My Drive/_faceswap_colab" if location == "GoogleDrive" else "."
_APPDIR=os.path.join(_ROOT, "faceswap")
_APP=os.path.join(_APPDIR, "faceswap.py")
fs_backend_path=os.path.join(_APPDIR, "config", ".faceswap")
!pip install tensorflow-gpu==1.15
!echo -en "\e[33m"
print("App Location:", _APP)
if app_update == "New" or not os.path.isfile(_APP):
  if os.path.isdir(_APPDIR):
    print("Uninstalling pre-existing app...")
    shutil.rmtree(_APPDIR)
  print("Getting Faceswap...")
  !echo -en "\e[0m"
  !git clone --depth 1 -b master https://github.com/deepfakes/faceswap.git "$_APPDIR"
  %cd "$_APPDIR"
  !echo -en "\e[33m"
  print("Installing Dependencies. This may take a minute or two...")
  !echo -en "\e[0m"
  !pip install -r "$_APPDIR/requirements.txt"
  !echo -en "\e[33m"
  print("Setting backend to nvidia:", fs_backend_path)
  with open(fs_backend_path, "w") as backend_config:
    backend_config.write("{\"backend\": \"nvidia\"}")
else:
  !echo -en "\e[33m"
  print("Updating Dependencies. This may take a minute or two...")
  !echo -en "\e[0m"
  %cd "$_APPDIR"
  !git pull 2>&1
  !echo -en "\e[33m"
  print("Setting backend to nvidia:", fs_backend_path)
  !echo -en "\e[0m"
  with open(fs_backend_path, "w") as backend_config:
    backend_config.write("{\"backend\": \"nvidia\"}")
  !python "$_APPDIR/update_deps.py"

!echo -en "\e[0m"



App Location: /content/drive/My Drive/_faceswap_colab/faceswap/faceswap.py
Uninstalling pre-existing app...
Getting Faceswap...
Cloning into '/content/drive/My Drive/_faceswap_colab/faceswap'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 261 (delta 56), reused 91 (delta 22), pack-reused 0
Receiving objects: 100% (261/261), 634.48 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Checking out files: 100% (251/251), done.
Installing Dependencies. This may take a minute or two...
Ignoring pywin32: markers 'sys_platform == "win32"' don't match your environment
Ignoring pynvx: markers 'sys_platform == "darwin"' don't match your environment
  Cloning https://github.com/deepfakes/nvidia-ml-py3.git to /tmp/pip-req-build-vft8qxeq
  Running command git clone -q https://github.com/deepfakes/nvidia-ml-py3.git /tmp/pip-req-build-vft8qxeq
  Created wheel for nvidia-ml-py3: filena

We're good to go! Now decide if you want to [Extract](#scrollTo=aWiYFqK4R64L), [Train](#scrollTo=o_M1CIMUr0hR) or [Convert](#scrollTo=_Kp5gjPqr0_x)


# Extract

Extract faces from a video or a folder of images

In [0]:
cliopts=!python "$_APP" extract -h
opts = dict()
for line in cliopts:
  line = line.strip()
  if line.startswith("-") and "--" in line:
    name = None
    choices = None
    line_opts = line.split()
    print("=============================")
    print(line_opts)
    opt = line_opts[0][1:].replace(",", "") 
    if line_opts[1].startswith("--"):
      name = line_opts[1].replace("--", "")
    elif line_opts[1].startswith("(") and line_opts[1].endswith(")"):
      choices=[choice for choice in line_opts[1][1:-1].split()]
    else:
      print(line)
    opts[opt]=dict(name=name, choices=choices)
    print(opts)


['-h,', '--help', 'show', 'this', 'help', 'message', 'and', 'exit']
{'h': {'name': 'help', 'choices': None}}
['-C', 'CONFIGFILE,', '--configfile', 'CONFIGFILE']
-C CONFIGFILE, --configfile CONFIGFILE
{'h': {'name': 'help', 'choices': None}, 'C': {'name': None, 'choices': None}}
['-L', '{INFO,VERBOSE,DEBUG,TRACE},', '--loglevel', '{INFO,VERBOSE,DEBUG,TRACE}']
-L {INFO,VERBOSE,DEBUG,TRACE}, --loglevel {INFO,VERBOSE,DEBUG,TRACE}
{'h': {'name': 'help', 'choices': None}, 'C': {'name': None, 'choices': None}, 'L': {'name': None, 'choices': None}}
['-LF', 'LOGFILE,', '--logfile', 'LOGFILE']
-LF LOGFILE, --logfile LOGFILE
{'h': {'name': 'help', 'choices': None}, 'C': {'name': None, 'choices': None}, 'L': {'name': None, 'choices': None}, 'LF': {'name': None, 'choices': None}}
['-i', 'INPUT_DIR,', '--input-dir', 'INPUT_DIR']
-i INPUT_DIR, --input-dir INPUT_DIR
{'h': {'name': 'help', 'choices': None}, 'C': {'name': None, 'choices': None}, 'L': {'name': None, 'choices': None}, 'LF': {'name': None,

# Train

Train a model

# Convert
Swap Faces on a trained model